In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
def get_orientations(df, fix='change', slow=True):
    from evaluate_data import integrate_yaw
    
    orientations = df.yaw_deg.values.copy()
    orientations[~np.isnan(orientations)] = np.unwrap(
        orientations[~np.isnan(orientations)] / 180 * np.pi) * 180 / np.pi

    if fix == 'change':
        direction = np.zeros(len(orientations) - 1)
        diff = orientations[1:] - orientations[:-1]
        direction[diff < 1e-1] = -1
        direction[diff > 1e-1] = 1
        change_direction = np.abs(direction[1:] - direction[:-1])

        start_idx = np.where(change_direction == 1)[0][0] + 2
        change_indices = np.where(change_direction == 2)[0] + 2
        if len(change_indices) != 2:
            print(f'did not find 2 direction changes!')
        else:
            orientations = np.zeros(len(orientations))
            increase = np.linspace(0, -360, change_indices[0] - start_idx)
            decrease = np.linspace(-360, 0,
                                   change_indices[1] - change_indices[0])
            orientations[start_idx:change_indices[0]] = increase
            orientations[change_indices[0]:change_indices[1]] = decrease
    elif fix == 360:
        if slow:
            start_idx = 30
            increase = 205
            pause = 55
            decrease = 205
        else:
            start_idx = 15
            increase = 132
            pause = 22
            decrease = 102
        start_angle = -0
        end_angle = -360
        idx = start_idx
        orientations[:idx] = start_angle
        orientations[idx:idx + increase] = np.linspace(start_angle, end_angle,
                                                       increase)
        idx += increase
        orientations[idx:idx + pause] = end_angle
        idx += pause
        orientations[idx:idx + decrease] = np.linspace(end_angle, start_angle,
                                                       decrease)
        idx += decrease
        orientations[idx:] = start_angle
    elif fix == 90:
        start_idx = 30
        deg90 = 370
        orientations[:start_idx] = 0
        orientations[start_idx:start_idx + deg90] = -90
        orientations[start_idx + deg90:] = 0
    elif fix == 0:
        orientations = np.zeros(len(orientations))
    elif fix == 'rate':
        orientations = integrate_yaw(df.timestamp.values,
                                     df.yaw_rate_deg.values)

    elif fix is None:
        pass
    else:
        raise ValueError(fix)

    return orientations

In [ ]:
from evaluate_data import read_df, get_spec, get_positions
from dynamic_analysis import add_pose_to_df

distance = None
#exp_name = '2020_10_30_dynamic'; degree=0; start_idx = 0# correct mic positions, but only one degree
#exp_name = '2020_10_30_dynamic_test'; degree=0; start_idx=0 #degree=90; 
#exp_name = '2020_10_30_dynamic_move'; degree=0; start_idx=0
#exp_name = '2020_11_03_sweep_old'; degree=90; start_idx = 20
#exp_name = '2020_11_03_sweep'; degree=90; start_idx = 20

#exp_name = '2020_11_10_buzzer'; degree=45; start_idx = 20
#exp_name = '2020_11_10_buzzer'; degree=0; start_idx = 20
#exp_name = '2020_11_10_buzzer'; degree=0; start_idx = 20

#exp_name = '2020_11_12_speaker360'; degree=360; start_idx = 20
#degree_offset = 15 #

#exp_name = '2020_11_13_speaker360'; degree=360; start_idx = 20
#degree_offset = 230 #

exp_name = '2020_11_23_wall2'; degree=360; start_idx = 20; distance=0
degree_offset = 0 #

params = dict(
    distance = distance,
    degree = degree,
    props = False,
    snr = False,
    motors = True,
    source = 'mono4125',
    exp_name = exp_name
)
title = f"motors: {params['motors']}, degree: {params['degree']}"

df, df_pos = read_df(**params)
df.head()

df = df.iloc[start_idx:]
index_start = df.iloc[0]['index']
df_pos = df_pos.loc[df_pos.index >= index_start]
add_pose_to_df(df, df_pos)
#df.head()

frequencies_matrix = np.array([*df.loc[:,'frequencies']])
plt.figure()
plt.title('frequency selection over time')
plt.pcolormesh(frequencies_matrix.T)
plt.colorbar()

In [ ]:
csv_file = '../experiments/2020_11_13_speaker360/Yaw-20201113T16-29-16.csv'
df_log = pd.read_csv(csv_file)
print(df_log.head())

integrated_yaw = integrate_yaw(df_log.Timestamp.values, df_log['gyro.z'].values)

fig = plt.figure()
fig.set_size_inches(10, 5)
plt.plot(df_log.Timestamp, df_log['gyro.z'])
plt.plot(df_log.Timestamp, df_log['stabilizer.yaw'])
plt.plot(df_log.Timestamp, integrated_yaw)
plt.xlim([35000, 70000])

In [ ]:
times_all = (df.audio_timestamp.values - df.iloc[0].audio_timestamp) / 1e6

orientations = get_orientations(df, fix=None)
orientations_hard = get_orientations(df, fix=degree, slow=False)
orientations_change = get_orientations(df, fix='change')
orientations_rate = get_orientations(df, fix='rate')

plt.figure()
plot_times = range(len(times_all))
plt.plot(plot_times, orientations, label='raw')
plt.plot(plot_times, orientations_hard, label=degree)
plt.plot(plot_times, orientations_change, label='change')
plt.plot(plot_times, orientations_rate, label='rate')
plt.grid()
plt.xlabel('time [s]')
plt.ylabel('yaw [deg]')
plt.legend()

In [ ]:
from audio_stack.beam_former import BeamFormer, normalize_rows, combine_rows
import progressbar

bin_selection = "fixed"; selected_hz = [4100]

#columns = range(len(df))
columns = list(range(len(df)))[::3]

selected_idx = list(set([np.argmin(np.abs(df.iloc[0].frequencies - hz)) for hz in selected_hz]))
print(selected_idx)
print('frequency bin:', df.iloc[0].frequencies[selected_idx])
print('desired:', selected_hz)
    
# covariance averaging
#averaging = 'iir'
#alpha = 0.5 # set to 1 for no effect

#orientations = orientations_hard[columns]
orientations = orientations_rate[columns]
times = times_all[columns]

start_orientation = orientations[~np.isnan(orientations)][0]
raw_orientations =  start_orientation - orientations
abs_orientations = np.full(orientations.shape, start_orientation)
    
raw_orientations += degree_offset
abs_orientations += degree_offset
raw_orientations[raw_orientations < 0] += 360
abs_orientations[abs_orientations < 0] += 360
raw_orientations[raw_orientations >= 360] -= 360
abs_orientations[abs_orientations >= 360] -= 360

In [ ]:
signals_f = np.array([*df['signals_f']]) # n_times x n_mics x n_freqs
print(signals_f.shape[2])

fig, (ax, ax_g) = plt.subplots(2)
ax_g.plot(times, orientations)
fig, axs = plt.subplots(signals_f.shape[1] + 1, sharex=True)
axs[-1].plot(times, orientations)
fig.suptitle(title)
for i in range(signals_f.shape[1]):
    for j in range(signals_f.shape[2]):
        axs[i].plot(times_all, 
                    np.sum(np.abs(signals_f[:, i, [j]]), axis=1), 
                    color=f'C{i}', alpha=0.1)
        
        ax.plot(times_all, 
                    np.sum(np.abs(signals_f[:, i, [j]]), axis=1), 
                    color=f'C{i}', alpha=0.1)
    axs[i].plot(times_all, 
            np.sum(np.abs(signals_f[:, i, selected_idx])/len(selected_idx), axis=1),
            color=f'C{i}',
            label=f'mic{i}')
    ax.plot(times_all, 
            np.sum(np.abs(signals_f[:, i, selected_idx])/len(selected_idx), axis=1),
            color=f'C{i}',
            label=f'mic{i}')
axs[i].set_xlabel('time [s]')
#ax.set_ylim([1e-3, 4])
#ax.legend([f"mic{i}" for i in range(row.signals_f.shape[0])])
ax.legend()
ax.set_xlabel('time [s]')
ax.set_title(title)

# test different mic combinations

In [ ]:
def plot_heatmap(ax):
    extra_kwargs = dict(
        color='red',
        lw=2
    )
    angles = BeamFormer.theta_scan * 180 / np.pi

    #plot_times = range(len(times))
    plot_times = times
    ax.pcolormesh(plot_times, angles, raw_heatmap)
    ax.plot(plot_times, raw_orientations, **extra_kwargs)
    ax.set_ylim(0, 360)
    ax.set_xlabel('time [s]')
    ax.set_ylabel('angle [deg]')

In [ ]:
import itertools

combination_n = 5
combination_method = "sum"
normalization_method = "none"
method = "das"
averaging = 'none'
alpha = 0.5
n_columns = len(columns)

for num in [4]: #[2, 3, 4]:
    combis = list(itertools.combinations(range(4), num))
    n_combis = len(combis)
    
    fig, axs = plt.subplots(1, n_combis, sharey=True, sharex=True, squeeze=False)
    axs = axs.reshape((n_combis,))
    fig.suptitle(title)
    fig.set_size_inches(15, 5)
    axs_i = 0

    for mic_selection in combis:
        mic_positions = df.iloc[0].mic_positions[mic_selection, :]
        beam_former = BeamFormer(mic_positions=mic_positions)

        R_idx = 0; R_list = [[]] * combination_n
        R = None
    
        angles = BeamFormer.theta_scan
        raw_heatmap = np.zeros((len(angles), n_columns))

        i_col = 0
        dynamic_spectrum = None
        multi_spectrum = None

        with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
            for i in columns:
                row = df.iloc[i]

                if bin_selection == "fixed":
                    signals_f = row.signals_f[:, selected_idx] # n_mics x n_freqs
                    signals_f = signals_f[mic_selection, :]
                    freqs = row.frequencies[selected_idx]
                else:
                    # TODO(FD): implement this
                    pass

                orientation_deg = row.yaw_deg
                timestamp = row.timestamp

                R_new = beam_former.get_correlation(signals_f.T) # n_freqs x n_mics x n_mics
                if averaging == 'iir':
                    if R is None:
                        R = R_new
                    else:
                        R = alpha*R_new + (1-alpha)*R
                elif averaging == 'ma':
                    R_list[R_idx] = R_new
                    R_idx = (R_idx + 1) % combination_n
                    R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)
                elif averaging == 'none':
                    R = R_new

                if method == 'mvdr':
                    raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
                elif method == 'das':
                    raw_spectrum = beam_former.get_das_spectrum(R, freqs)

                time_sec = row.audio_timestamp / 1e6

                #### RAW
                raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
                raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")

                raw_heatmap[:, i_col] = raw_spectrum.flatten()
                if dynamic_spectrum is not None:
                    dynamic_heatmap[:, i_col] = dynamic_spectrum.flatten()
                if multi_spectrum is not None:
                    multi_heatmap[:, i_col] = multi_spectrum.flatten()
                i_col += 1

                if i_col >= n_columns:
                    break

                p.update(i_col)

        plot_heatmap(axs[axs_i])
        axs[axs_i].set_title(mic_selection)
        axs_i += 1

    #fig.savefig('figures/{exp_name}_mic_selection.png')

# test different dynamic schemes

In [ ]:
def plot_results(fig, axs):
    axs = axs.reshape((1, 3))
    fig.set_size_inches(15, 5)

    extra_kwargs = dict(
    color='red',
    lw=2
    )

    angles = BeamFormer.theta_scan * 180 / np.pi

    axs[0, 0].pcolormesh(times, angles, raw_heatmap)
    axs[0, 0].plot(times, raw_orientations, **extra_kwargs)
    axs[0, 0].set_title('raw heatmap')

    axs[0, 1].pcolormesh(times, angles, multi_heatmap)
    axs[0, 1].plot(times, abs_orientations, **extra_kwargs)
    axs[0, 1].set_title('multi heatmap')

    axs[0, 2].pcolormesh(times, angles, dynamic_heatmap)
    axs[0, 2].plot(times, abs_orientations, **extra_kwargs)
    axs[0, 2].set_title('dynamic heatmap')

    axs[0, 0].set_ylabel('angle [deg]')
    axs[-1, 0].set_xlabel('time [s]')
    axs[-1, 1].set_xlabel('time [s]')
    axs[-1, 2].set_xlabel('time [s]')
    axs[0, 0].set_ylim(0, 360)

In [ ]:
import itertools

mic_selection = range(4)

mic_positions = df.iloc[0].mic_positions[mic_selection, :]
beam_former = BeamFormer(mic_positions=mic_positions)

if bin_selection == "fixed":
    selected_idx = [np.argmin(np.abs(df.iloc[0].frequencies - hz)) for hz in selected_hz]
    frequencies = df.iloc[0].frequencies[selected_idx] 
    beam_former.init_dynamic_estimate(frequencies, combination_n, combination_method, normalization_method)
    beam_former.init_multi_estimate(frequencies, combination_n)
else:
    # TODO(FD): implement good initialization
    pass


R_idx = 0; R_list = [[]] * combination_n
R = None

n_columns = len(columns)
angles = BeamFormer.theta_scan

raw_heatmap = np.zeros((len(angles), n_columns))
dynamic_heatmap = np.zeros((len(angles), n_columns))
multi_heatmap = np.zeros((len(angles), n_columns))

i_col = 0
dynamic_spectrum = None
multi_spectrum = None

with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
    for i in columns:
        row = df.iloc[i]

        if bin_selection == "fixed":
            signals_f = row.signals_f[:, selected_idx] # n_mics x n_freqs
            signals_f = signals_f[mic_selection, :]
            freqs = row.frequencies[selected_idx]
        else:
            # TODO(FD): implement this
            pass

        orientation_deg = orientations[i_col] #row.yaw_deg
        timestamp = row.timestamp

        R_new = beam_former.get_correlation(signals_f.T) # n_freqs x n_mics x n_mics
        if averaging == 'iir':
            if R is None:
                R = R_new
            else:
                R = alpha*R_new + (1-alpha)*R
        elif averaging == 'ma':
            R_list[R_idx] = R_new
            R_idx = (R_idx + 1) % combination_n
            R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)
        elif averaging == 'none':
            R = R_new

        if method == 'mvdr':
            raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
        elif method == 'das':
            raw_spectrum = beam_former.get_das_spectrum(R, freqs)
        time_sec = row.audio_timestamp / 1e6

        if not pd.isna(orientation_deg):
            #### DYNAMIC
            beam_former.add_to_dynamic_estimates(raw_spectrum, orientation_deg=orientation_deg)
            dynamic_spectrum = beam_former.get_dynamic_estimate()
            dynamic_spectrum = combine_rows(dynamic_spectrum, combination_method, keepdims=True)
            dynamic_spectrum = normalize_rows(dynamic_spectrum, method="zero_to_one")

            #### MULTI
            beam_former.add_to_multi_estimate(signals_f.T, freqs, time_sec, orientation_deg)
            multi_spectrum = beam_former.get_multi_estimate(method=method)
            multi_spectrum = combine_rows(multi_spectrum, combination_method, keepdims=True)
            multi_spectrum = normalize_rows(multi_spectrum, method="zero_to_one")
        else:
            print('not updating dynamic and multi:', i_col)

        #### RAW
        raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
        raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")

        raw_heatmap[:, i_col] = raw_spectrum.flatten()
        if dynamic_spectrum is not None:
            dynamic_heatmap[:, i_col] = dynamic_spectrum.flatten()
        if multi_spectrum is not None:
            multi_heatmap[:, i_col] = multi_spectrum.flatten()
        i_col += 1

        if i_col >= n_columns:
            break

        p.update(i_col)

    fig, axs = plt.subplots(1, 3, sharey=True, sharex=True)
    plot_results(fig, axs)

# Plotting (can start here directly!)

## run dynamic_analaysis.py to generate results

In [ ]:
from audio_stack.beam_former import BeamFormer

#dict(exp_name = '2020_10_30_dynamic_test', degree=0, motors=''),
#dict(exp_name = '2020_10_30_dynamic_test', degree=90, motors=''),
#dict(exp_name = '2020_10_30_dynamic_move', degree=0, motors='')
#dict(exp_name = '2020_11_03_sweep', degree=90, motors=''),
#dict(exp_name = '2020_11_03_sweep_old', degree=90, motors=''),

plots = [
    dict(exp_name = '2020_11_10_buzzer'),
]

BeamFormer()

for plot_dict in plots:
    exp_name = plot_dict.get('exp_name')
    fname = f'DynamicAnalaysis_{exp_name}.pkl'
    df_results = pd.read_pickle(fname)
    print('read', fname)
    
    df_raw = df_results.loc[(df_results.exp_name==exp_name)]
    for (motors, degree, combination_n, method, averaging), df_chosen in df_raw.groupby(['motors', 'degree', 'combination_n', 'method', 'covariance_averaging']):
        
        assert len(df_chosen) == 1, len(df_chosen)
        
        df_chosen=df_chosen.iloc[0]
        
        if method == 'mvdr':
            continue
        if combination_n == 2:
            continue

        plot_dict['motors'] = motors
        plot_dict['degree'] = degree
        plot_dict['combination_n'] = combination_n
        plot_dict['method'] = method
        plot_dict['averaging'] = averaging

        times = np.arange(df_chosen.raw_heatmap.shape[1])
        angles = BeamFormer.theta_scan * 180 / np.pi

        fig, axs = plt.subplots(1, 3)
        fig.set_size_inches(15, 5)
        axs[0].pcolormesh(times, angles, df_chosen.raw_heatmap)
        axs[0].set_title('raw heatmap')

        axs[1].pcolormesh(times, angles, df_chosen.multi_heatmap)
        axs[1].set_title('multi heatmap')

        axs[2].pcolormesh(times, angles, df_chosen.dynamic_heatmap)
        axs[2].set_title('dynamic heatmap')

        axs[0].set_ylabel('angle [deg]')
        fig.suptitle(plot_dict)